# KNN Models

## Objective

The objective of this notebook is to train and test different KNN models, by changing their hyperparameters, in order to obtain the best Random Forest model.
<br><br>
As discussed in "basic_models.ipynb", the transformations that will be used are Minimum and Geometric.

## Loading libraries and data

In [8]:
# importing important libraries

# transformations library
from transformations import minimum, geometric

# models
from sklearn.neighbors import KNeighborsClassifier

# loading data
import pickle

# other modules
from sklearn.model_selection import cross_val_score
from sklearn.metrics import recall_score, make_scorer
from sklearn.model_selection import GridSearchCV
import numpy as np

In [2]:
# get base_dataset
data_path = "TrainTestData/train_data.pickle"
data = pickle.load(open(data_path, "rb"))

In [3]:
# function that calculates weighted_accuracy
# weights are basead on the frequency of the letters in the portuguese alphabet 
# source: https://pt.wikipedia.org/wiki/Alfabeto_portugu%C3%AAs#Frequ%C3%AAncia_da_ocorr%C3%AAncia_de_letras
# H, K, J, X and Z are not present
LETTERS_FREQUENCY = [
    14.63,
    1.04,
    3.88,
    5.01,
    12.57,
    1.02,
    1.30,
    6.18,
    2.78,
    4.74,
    5.05,
    10.73,
    2.52,
    1.20,
    6.53,
    7.81,
    4.34,
    4.63,
    1.67,
    0.01,
    0.01,
]
def weighted_accuracy(y_true, y_pred):
    recall_array = recall_score(y_true, y_pred, average=None)
    weights_total = 0
    result = 0
    for recall, weight in zip(recall_array, LETTERS_FREQUENCY):
        weights_total += weight
        result += recall * weight
    return result / weights_total
weighted_accuracy_score = make_scorer(weighted_accuracy)

## Choosing hyperparameters and transformations

In [4]:
# Minumum transformation
minimum_X = []
for observation in data["features"]:
    minimum_X.append(minimum(observation))

# Geometric transformation
geometric_X = []
for observation in data["features"]:
    geometric_X.append(geometric(observation))

In [18]:
# hyperparameters for first Grid Search
param_grid  = {
    "n_neighbors": [3, 5, 10],
    "p": [2, 5, 7]
}

## First Grid Search

In [19]:
# Minimum transformation
knn = KNeighborsClassifier()
grid_search_minimum = GridSearchCV(knn, param_grid, cv=5, scoring=weighted_accuracy_score, return_train_score=True)

grid_search_minimum.fit(minimum_X, data["labels"])

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [3, 5, 10], 'p': [2, 5, 7]},
             return_train_score=True, scoring=make_scorer(weighted_accuracy))

In [20]:
cvres = grid_search_minimum.cv_results_ 
results = sorted(zip(cvres["mean_test_score"], cvres["params"]), reverse=True)
for mean_score, params in results:
    print(mean_score, params)

0.8044002438457479 {'n_neighbors': 3, 'p': 2}
0.7878744283061077 {'n_neighbors': 5, 'p': 2}
0.771609962306422 {'n_neighbors': 3, 'p': 5}
0.7642929452214628 {'n_neighbors': 3, 'p': 7}
0.7530143449763355 {'n_neighbors': 5, 'p': 5}
0.7473544852921125 {'n_neighbors': 10, 'p': 2}
0.7415061952116215 {'n_neighbors': 5, 'p': 7}
0.7105776140317632 {'n_neighbors': 10, 'p': 5}
0.7034829770453674 {'n_neighbors': 10, 'p': 7}


In [21]:
# Geometric transformation
knn = KNeighborsClassifier()
grid_search_geometric= GridSearchCV(knn, param_grid, cv=5, scoring=weighted_accuracy_score, return_train_score=True)

grid_search_geometric.fit(geometric_X, data["labels"])

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [3, 5, 10], 'p': [2, 5, 7]},
             return_train_score=True, scoring=make_scorer(weighted_accuracy))

In [23]:
cvres = grid_search_geometric.cv_results_ 
results = sorted(zip(cvres["mean_test_score"], cvres["params"]), reverse=True)
for mean_score, params in results:
    print(mean_score, params)

0.9355476941454149 {'n_neighbors': 3, 'p': 7}
0.9311482571048174 {'n_neighbors': 3, 'p': 5}
0.92385766392811 {'n_neighbors': 5, 'p': 7}
0.9212556112985505 {'n_neighbors': 5, 'p': 5}
0.9199192497911461 {'n_neighbors': 3, 'p': 2}
0.9031644919750518 {'n_neighbors': 10, 'p': 7}
0.9010844097336432 {'n_neighbors': 5, 'p': 2}
0.9009482229005681 {'n_neighbors': 10, 'p': 5}
0.8772183473271602 {'n_neighbors': 10, 'p': 2}


The best model is clearly the KNN with n_neighbors=3 and p =7, using geometric transformation.
<br><br>
The best models seem to be the ones with low values of n_neighbors, so no real reason to test new values of it. On the other hand, the best p value was the highest one tested, so it's probably good to do a new search using higher valus of it.
<br><br>
Since this search is smaller, we will also test different types of weights (uniform or distance).

## Second Grid Search

In [24]:
# hyperparameters for second Grid Search
param_grid  = {
    "n_neighbors": [3],
    "p": [7, 9, 11],
    "weights": ["uniform", "distance"]
}

In [26]:
# Geometric transformation
knn = KNeighborsClassifier(n_jobs=-1)
grid_search_geometric= GridSearchCV(knn, param_grid, cv=5, scoring=weighted_accuracy_score, return_train_score=True, n_jobs=-1)

grid_search_geometric.fit(geometric_X, data["labels"])

GridSearchCV(cv=5, estimator=KNeighborsClassifier(n_jobs=-1), n_jobs=-1,
             param_grid={'n_neighbors': [3], 'p': [7, 9, 11],
                         'weights': ['uniform', 'distance']},
             return_train_score=True, scoring=make_scorer(weighted_accuracy))

In [27]:
cvres = grid_search_geometric.cv_results_ 
results = sorted(zip(cvres["mean_test_score"], cvres["params"]), reverse=True)
for mean_score, params in results:
    print(mean_score, params)

0.9417835703342794 {'n_neighbors': 3, 'p': 11, 'weights': 'distance'}
0.9416560533660521 {'n_neighbors': 3, 'p': 7, 'weights': 'distance'}
0.9415678389122464 {'n_neighbors': 3, 'p': 9, 'weights': 'distance'}
0.9355476941454149 {'n_neighbors': 3, 'p': 7, 'weights': 'uniform'}
0.9349337354609386 {'n_neighbors': 3, 'p': 11, 'weights': 'uniform'}
0.9344355279364747 {'n_neighbors': 3, 'p': 9, 'weights': 'uniform'}


Again, the best model is the one with the highest p value. We can also see that using weights=distance is better than uniform. Therefore, we will use this as a fixed hyperparameter in the next grid search.

## Third Grid Search

In [28]:
# hyperparameters for third Grid Search
param_grid  = {
    "n_neighbors": [3, 5, 10],
    "p": [11, 15, 19],
    "weights": ["distance"]
}

In [29]:
# Geometric transformation
knn = KNeighborsClassifier(n_jobs=-1)
grid_search_geometric= GridSearchCV(knn, param_grid, cv=5, scoring=weighted_accuracy_score, return_train_score=True, n_jobs=-1)

grid_search_geometric.fit(geometric_X, data["labels"])

GridSearchCV(cv=5, estimator=KNeighborsClassifier(n_jobs=-1), n_jobs=-1,
             param_grid={'n_neighbors': [3, 5, 10], 'p': [11, 15, 19],
                         'weights': ['distance']},
             return_train_score=True, scoring=make_scorer(weighted_accuracy))

In [30]:
cvres = grid_search_geometric.cv_results_ 
results = sorted(zip(cvres["mean_test_score"], cvres["params"]), reverse=True)
for mean_score, params in results:
    print(mean_score, params)

0.9419720876415763 {'n_neighbors': 3, 'p': 19, 'weights': 'distance'}
0.9417835703342794 {'n_neighbors': 3, 'p': 11, 'weights': 'distance'}
0.9417565320691814 {'n_neighbors': 3, 'p': 15, 'weights': 'distance'}
0.9382113254555648 {'n_neighbors': 5, 'p': 15, 'weights': 'distance'}
0.9381687320751372 {'n_neighbors': 5, 'p': 11, 'weights': 'distance'}
0.9369409888719 {'n_neighbors': 5, 'p': 19, 'weights': 'distance'}
0.9300938259027067 {'n_neighbors': 10, 'p': 15, 'weights': 'distance'}
0.9298930833168677 {'n_neighbors': 10, 'p': 19, 'weights': 'distance'}
0.9285312467057187 {'n_neighbors': 10, 'p': 11, 'weights': 'distance'}


Again, the highest p value is the best. However, since the difference in performance from it to the previous values is quite small, we will stop it here.

### Analysing Time performance

In [33]:
# average time per prediction
from time import time
best_knn = grid_search_geometric.best_estimator_
best_knn.fit(geometric_X, data["labels"])

start = time()
best_knn.predict(geometric_X)
end = time()
print((end - start) / len(geometric_X))

0.00168169891012126


## Conclusion

The best model uses n_neighbors=3, p=19 and weights="distance", with a performance of 94.20%.
<br><br>
The average time per prediction is 0.00168 seconds.